In [1]:
import tensorflow as tf
import requests 
import json
import base64
import random

from modules.utils import create_int_feature, create_float_feature, create_str_feature
from typing import Dict

In [2]:
def prepare_json(inputs: Dict):
    feature_spec = {}
    
    for key, value in inputs.items():
        if isinstance(value, float):
            feature_spec[key] = create_float_feature(value)
        elif isinstance(value, str):
            feature_spec[key] = create_str_feature(value)
        else:    
            feature_spec[key] = create_int_feature(value)
        
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({"instances": result})

In [3]:
def make_prediction(inputs: Dict, endpoint: str):
    json_data = prepare_json(inputs)
    
    endpoint = endpoint
    response = requests.post(endpoint, data=json_data)
    
    prediction = response.json()["predictions"]
    
    return prediction

In [6]:
# collaborative filtering model predict

cf_pred = make_prediction(
    inputs={"movieId": 10, "userId": 1},
    endpoint="http://localhost:8080/v1/models/movies-cf-model:predict",
)
cf_pred[0][0]

3.13760757

In [ ]:
# content-based filtering model predict

cbf_pred = make_prediction(
    inputs={"title": "Ngeri Ngeri Sedap", "userId": 1000, "genres": "Comedy"},
    endpoint="http://localhost:8081/v1/models/movies-cbf-model:predict",
)
cbf_pred[0][0]